# Mine

In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools import extractor
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import crystalball
import matplotlib.pyplot as plt
from hist import Hist
from coffea.analysis_tools import PackedSelection
from scipy.integrate import quad

In [2]:
#TODO: fix path issue
fname = "NANOAOD-1500_900.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
    delayed=False
).events()

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [ ]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [ ]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet")
events = events[mask]

jetOrdinality,jetPT,jetEta,jetPhi,jetBScore,m3M,m3PT,m3Eta,m3Phi,m4M,m4PT,m4Eta,m4Phi,isStopMatched,isChiMatched,isOther

In [ ]:
jets = events.Jet
gj = events.good_jets

In [ ]:
ak.local_index(gj, axis=1)

<Array [[0, 1, 2, 3, 4, 5], [...], ..., [0, 1, 2, 3]] type='4469 * var * int64'>

In [ ]:
def pad(arr):
            return ak.fill_none(
                ak.pad_none(arr, 100, axis=1, clip=True),
                0.0,
            )

In [ ]:
imap = {
    "ordinality": ak.local_index(gj, axis=1),
    "pt": gj.pt,
    "eta": gj.eta,
    "phi": gj.phi,
    "bscore": gj.btagDeepFlavB,
    "m3_m": gj[:, :3].sum().mass,
    "m3_pt": gj[:, :3].sum().pt,
    "m3_eta": gj[:, :3].sum().eta,
    "m3_phi": gj[:, :3].sum().phi,
    "m4_m": gj[:, :4].sum().mass,
    "m4_pt": gj[:, :4].sum().pt,
    "m4_eta": gj[:, :4].sum().eta,
    "m4_phi": gj[:, :4].sum().phi
}

In [ ]:
ordinality = ak.local_index(gj, axis=1)
pt = gj.pt
eta = gj.eta
phi = gj.phi
bscore = gj.btagDeepFlavB
m3_m = gj[:, :3].sum().mass
m3_pt = gj[:, :3].sum().pt
m3_eta = gj[:, :3].sum().eta
m3_phi = gj[:, :3].sum().phi
m4_m = gj[:, :4].sum().mass
m4_pt = gj[:, :4].sum().pt
m4_eta = gj[:, :4].sum().eta
m4_phi = gj[:, :4].sum().phi

In [ ]:
ak.zip(imap)[0]

<Array [{ordinality: 0, pt: 448, ...}, ...] type='6 * {ordinality: int64, p...'>

In [ ]:
(imap["m4_m"])[k]

1412.2499

In [ ]:
imap

{'ordinality': <Array [[0, 1, 2, 3, 4, 5], [...], ..., [0, 1, 2, 3]] type='4469 * var * int64'>,
 'pt': <Array [[448, 359, 272, ..., 43.4, 39.8], ...] type='4469 * var * float32[p...'>,
 'eta': <Array [[0.138, -0.454, ..., -0.3, 2.05], ...] type='4469 * var * float32[p...'>,
 'phi': <Array [[-2.74, -0.685, ..., -0.858], ...] type='4469 * var * float32[param...'>,
 'bscore': <Array [[0.0232, 0.717, ..., 0.0298], ...] type='4469 * var * float32[param...'>,
 'm3_m': <Array [1.15e+03, 1.14e+03, ..., 1.06e+03, 2.6e+03] type='4469 * float32'>,
 'm3_pt': <Array [164, 228, 132, 112, 40, ..., 102, 58.2, 103, 265] type='4469 * float32'>,
 'm3_eta': <Array [0.25, 1.96, -0.841, 2.66, ..., 2.83, -1.03, 1.2] type='4469 * float32'>,
 'm3_phi': <Array [-2.18, 2.5, -1.3, ..., -1.56, 0.208, 0.0621] type='4469 * float32'>,
 'm4_m': <Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4469 * float32'>,
 'm4_pt': <Array [110, 82.2, 99.2, 115, ..., 94.6, 48, 104, 111] type='4469 * float32'>,
 'm4_eta'

# What

In [14]:
def pad(arr):
            return ak.fill_none(
                ak.pad_none(arr, 100, axis=1, clip=True),
                0.0,
            )

In [31]:
(), {
            "points": ak.values_astype(retmap["points"], "float32"),
            "features": ak.values_astype(retmap["features"], "float32"),
            "mask": ak.values_astype(retmap["mask"], "float16"),
        }

KeyError: 'features'

In [25]:
retmap

{'points': <Array [[[0.138, -0.454, ..., 0, 0]], ...] type='4469 * 1 * 100 * float64'>,
 'features': <Array [[[-2.74, -0.685, ..., 0, 0]], ...] type='4469 * 1 * 100 * float64'>,
 'mask': <Array [[[1, 1, 1, 1, ..., 0, 0, 0, 0]], ...] type='4469 * 1 * 100 * float64'>}

# Devin's

In [3]:
from coffea.ml_tools.torch_wrapper import torch_wrapper
from analyzer.core import analyzerModule
from analyzer.matching import object_matching
import awkward as ak

In [4]:
class jetAssignmentNN(torch_wrapper):
    def prepare_awkward(self,events):

        awk = self.get_awkward_lib(events)
        #jets = ak.flatten(events.good_jets)
        jets = events.good_jets
        m3 = jets[:,1:4].sum()
        m4 = jets[:,0:4].sum()

        imap = {
            "features": {
                "jetOrdinality":	ak.local_index(jets),
                "jetPT": 		jets.pt,
                "jetEta": 		jets.eta,
                "jetPhi": 		jets.phi,
                "jetBScore": 		jets.btagDeepFlavB,
                "m3M": 			m3.mass,
                "m3PT": 		m3.pt,
                "m3Eta": 		m3.eta,
                "m3Phi": 		m3.phi,
                "m4M": 			m4.mass,
                "m4PT":			m4.pt,
                "m4Eta":		m4.eta,
                "m4Phi":		m4.phi,
            }
        }

        retmap = {
            k: ak.concatenate([x[:, np.newaxis, :] for x in imap[k].values()], axis=1)
            for k in imap.keys()
        }
        
        return(), {
            "features": awk.values_astype(imap["features"],"float32")
        }

In [16]:
jets = events.good_jets
m3 = jets[:,1:4].sum()
m4 = jets[:,0:4].sum()

imap = {
    "features": {
        "jetOrdinality":	ak.local_index(jets),
        "jetPT": 		jets.pt,
        "jetEta": 		jets.eta,
        "jetPhi": 		jets.phi,
        "jetBScore": 		jets.btagDeepFlavB,
        "m3M": 			m3.mass,
        "m3PT": 		m3.pt,
        "m3Eta": 		m3.eta,
        "m3Phi": 		m3.phi,
        "m4M": 			m4.mass,
        "m4PT":			m4.pt,
        "m4Eta":		m4.eta,
        "m4Phi":		m4.phi,
    }
}

In [28]:
imap['features']["m4M"]

<Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4469 * float32'>

In [29]:
retmap = {
   "features": ak.concatenate([x[:, np.newaxis] for x in imap["features"].values()], axis=1)
}

In [22]:
retmap

{'features': <Array [[??, ??, ??, ??, ..., ??, ??, ??], ...] type='4469 * 13 * union[var...'>}

# Also devin

In [85]:
class jetAssignmentNN(torch_wrapper):
    def prepare_awkward(self,events):
        awk = self.get_awkward_lib(events)
        
        jets = events.good_jets
        flat_jets = ak.flatten(jets)

        m3 = jets[:,1:4].sum()
        m4 = jets[:,0:4].sum()

        ones = ak.ones_like(jets.pt)
        ones_flat = ak.ones_like(flat_jets.pt)

        imap = {
            "features": {
                "jetOrdinality":	ak.flatten(ak.local_index(jets, axis=1)),
                "jetPT": 		flat_jets.pt,
                "jetEta": 		flat_jets.eta,
                "jetPhi": 		flat_jets.phi,
                "jetBScore": 	flat_jets.btagDeepFlavB,
                "m3M": 			ak.flatten(ones * m3.mass),
                "m3PT": 		ak.flatten(ones * m3.pt),
                "m3Eta": 		ak.flatten(ones * m3.eta),
                "m3Phi": 		ak.flatten(ones * m3.phi),
                "m4M": 			ak.flatten(ones * m4.mass),
                "m4PT":			ak.flatten(ones * m4.pt),
                "m4Eta":		ak.flatten(ones * m4.eta),
                "m4Phi":	    ak.flatten(ones * m4.phi)
            }
        }

        retmap = {
            "features": ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)
        }

        print(retmap)
        
        return(),{
            "features": awk.values_astype(retmap["features"],"float32")
        }

In [60]:
ak.flatten(events.good_jets)

<JetArray [Jet, Jet, ..., Jet] type='21803 * Jet[area: float32[parameters={...'>

In [72]:
jets = events.good_jets
flat_jets = ak.flatten(jets)

m3 = jets[:,1:4].sum()
m4 = jets[:,0:4].sum()

ones = ak.ones_like(jets.pt)
ones_flat = ak.ones_like(flat_jets.pt)

imap = {
    "features": {
        "jetOrdinality":	ak.flatten(ak.local_index(jets, axis=1)),
        "jetPT": 		flat_jets.pt,
        "jetEta": 		flat_jets.eta,
        "jetPhi": 		flat_jets.phi,
        "jetBScore": 	flat_jets.btagDeepFlavB,
        "m3M": 			ak.flatten(ones * m3.mass),
        "m3PT": 		ak.flatten(ones * m3.pt),
        "m3Eta": 		ak.flatten(ones * m3.eta),
        "m3Phi": 		ak.flatten(ones * m3.phi),
        "m4M": 			ak.flatten(ones * m4.mass),
        "m4PT":			ak.flatten(ones * m4.pt),
        "m4Eta":		ak.flatten(ones * m4.eta),
        "m4Phi":	    ak.flatten(ones * m4.phi)
    }
}

In [73]:
vals = imap['features'].values()

In [74]:
vals

dict_values([<Array [0, 1, 2, 3, 4, 5, 0, 1, ..., 2, 3, 4, 0, 1, 2, 3] type='21803 * int64'>, <Array [448, 359, 272, 237, ..., 320, 202, 160] type='21803 * float32[param...'>, <Array [0.138, -0.454, ..., 0.747, -0.555] type='21803 * float32[parameters...'>, <Array [-2.74, -0.685, 1.42, ..., 2.83, 3.03] type='21803 * float32[paramet...'>, <Array [0.0232, 0.717, ..., 0.964, 0.621] type='21803 * float32[parameters=...'>, <Array [829, 829, 829, 829, ..., 938, 938, 938, 938] type='21803 * float32'>, <Array [455, 455, 455, 455, ..., 664, 664, 664, 664] type='21803 * float32'>, <Array [-0.041, -0.041, -0.041, ..., -1.08, -1.08] type='21803 * float32'>, <Array [0.648, 0.648, 0.648, ..., 2.72, 2.72, 2.72] type='21803 * float32'>, <Array [1.39e+03, 1.39e+03, ..., 2.81e+03, 2.81e+03] type='21803 * float32'>, <Array [110, 110, 110, 110, ..., 111, 111, 111, 111] type='21803 * float32'>, <Array [0.384, 0.384, 0.384, ..., 1.74, 1.74, 1.74] type='21803 * float32'>, <Array [2.03, 2.03, 2.03, ..., 0.312

# this here!!

In [88]:
net_input = ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)

In [80]:
imap["features"].values()

dict_values([<Array [0, 1, 2, 3, 4, 5] type='6 * int64'>, <Array [448, 359, 272, 237, 43.4, 39.8] type='6 * float32[parameters={"__do...'>, <Array [0.138, -0.454, 0.52, ..., -0.3, 2.05] type='6 * float32[parameters=...'>, <Array [-2.74, -0.685, ..., -2.57, -0.858] type='6 * float32[parameters={"_...'>, <Array [0.0232, 0.717, ..., 0.0811, 0.0298] type='6 * float32[parameters={"...'>, <Array [829, 829, 829, 829, 829, 829] type='6 * float32[parameters={"__doc_...'>, <Array [455, 455, 455, 455, 455, 455] type='6 * float32[parameters={"__doc_...'>, <Array [-0.041, -0.041, ..., -0.041, -0.041] type='6 * float32[parameters={...'>, <Array [0.648, 0.648, 0.648, ..., 0.648, 0.648] type='6 * float32[parameter...'>, <Array [1.39e+03, 1.39e+03, ..., 1.39e+03] type='6 * float32[parameters={"_...'>, <Array [110, 110, 110, 110, 110, 110] type='6 * float32[parameters={"__doc_...'>, <Array [0.384, 0.384, 0.384, ..., 0.384, 0.384] type='6 * float32[parameter...'>, <Array [2.03, 2.03, 2.03, ..., 2.03, 2.0

In [78]:
def addEventLevelVars(events):
    ht = ak.sum(events.good_jets.pt, axis=1)
    events["HT"] = ht
    return events

In [79]:
def deltaRMatch(events, analyzer):
    matched_jets, matched_quarks, dr, idx_j, idx_q, _ = object_matching(
        events.good_jets, events.SignalQuarks, 0.2, 0.5, True
    )
    # print(f"IndexQ: {idx_q}")
    # print(f"IndexJ: {idx_j}")
    # print(f"MQ: {matched_quarks}")
    # print(f"MJ: {matched_jets}")
    # _, _, _, ridx_q, ridx_j, _ = object_matching(
    #    events.SignalQuarks, events.good_jets, 0.3, 0.5, True
    # )
    events["matched_quarks"] = matched_quarks
    events["matched_jets"] = matched_jets
    events["matched_dr"] = dr
    events["matched_jet_idx"] = idx_j
    return events, analyzer

In [80]:
def addNNScores(events):
    model = jetAssignmentNN("jetMatcherNN.pt")
    scores = model(events)
    print(scores)
    events["NNStopProb"]  = scores[:,0]
    events["NNChiProb"]   = scores[:,1]
    events["NNOtherProb"] = scores[:,2] 
    return events

In [86]:
model = jetAssignmentNN("jetMatcherNN.pt")

In [87]:
model(events)

{'features': <Array [[0, 448, 0.138, ..., 0.384, 2.03], ...] type='21803 * 13 * float64'>}


/usr/local/lib/python3.10/site-packages/coffea/ml_tools/helper.py:175: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


RuntimeError: PytorchStreamReader failed locating file constants.pkl: file not found

In [11]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [12]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [13]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet")
events = events[mask]

In [49]:
addNNScores(events)

ValueError: too many values to unpack (expected 2)